<a href="https://colab.research.google.com/github/nss-17/Face-tracking-Emotion-detection-Eye-tracking/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
from keras.applications import MobileNet
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [14]:
# MobileNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224


In [15]:
MobileNet = MobileNet(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

In [16]:
for layer in MobileNet.layers:
    layer.trainable = True

# Let's print our layers
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer True
1 Conv2D True
2 BatchNormalization True
3 ReLU True
4 DepthwiseConv2D True
5 BatchNormalization True
6 ReLU True
7 Conv2D True
8 BatchNormalization True
9 ReLU True
10 ZeroPadding2D True
11 DepthwiseConv2D True
12 BatchNormalization True
13 ReLU True
14 Conv2D True
15 BatchNormalization True
16 ReLU True
17 DepthwiseConv2D True
18 BatchNormalization True
19 ReLU True
20 Conv2D True
21 BatchNormalization True
22 ReLU True
23 ZeroPadding2D True
24 DepthwiseConv2D True
25 BatchNormalization True
26 ReLU True
27 Conv2D True
28 BatchNormalization True
29 ReLU True
30 DepthwiseConv2D True
31 BatchNormalization True
32 ReLU True
33 Conv2D True
34 BatchNormalization True
35 ReLU True
36 ZeroPadding2D True
37 DepthwiseConv2D True
38 BatchNormalization True
39 ReLU True
40 Conv2D True
41 BatchNormalization True
42 ReLU True
43 DepthwiseConv2D True
44 BatchNormalization True
45 ReLU True
46 Conv2D True
47 BatchNormalization True
48 ReLU True
49 DepthwiseConv2D True
50 BatchNorma

In [17]:
def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)

    top_model = Dense(1024,activation='relu')(top_model)

    top_model = Dense(512,activation='relu')(top_model)

    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model


In [18]:
num_classes = 5

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

In [19]:
#train_data_dir = '/content/gdrive/MyDrive/Autism_model_training/autistic_detection_by_emotion/Dataset/trainData'
#validation_data_dir = '/content/gdrive/MyDrive/Autism_model_training/autistic_detection_by_emotion/Dataset/validationData'

train_data_dir = '/content/gdrive/MyDrive/autistic_detection_by_emotion/Dataset/trainData'
validation_data_dir = '/content/gdrive/MyDrive/autistic_detection_by_emotion/Dataset/validationData'


In [20]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

In [31]:
train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )
train_generator.samples

Found 250 images belonging to 5 classes.


250

In [33]:
validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')
validation_generator.samples

Found 250 images belonging to 5 classes.


250

In [23]:
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [24]:
checkpoint = ModelCheckpoint(
                             'emotion_face_mobilNet.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)


In [25]:
earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)


In [26]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=5,
                                            verbose=1,
                                            factor=0.2,
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 24176
nb_validation_samples = 3006

epochs = 25

In [32]:
#steps_per_epoch=nb_train_samples//batch_size,
history = model.fit_generator(
            train_generator,
            steps_per_epoch = (train_generator.samples)//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=(validation_generator.samples)//batch_size)

<ipython-input-32-55e6b98be179>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
7/7 [==============================] - ETA: 0s - loss: 0.0911 - accuracy: 0.9817
Epoch 1: val_loss did not improve from 2.66345


7/7 [==============================] - 6s 968ms/step - loss: 0.0911 - accuracy: 0.9817 - val_loss: 3.6960 - val_accuracy: 0.6607 - lr: 0.0010
Epoch 2/25
7/7 [==============================] - ETA: 0s - loss: 0.2130 - accuracy: 0.9358
Epoch 2: val_loss improved from 2.66345 to 0.78321, saving model to emotion_face_mobilNet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 5s 781ms/step - loss: 0.2130 - accuracy: 0.9358 - val_loss: 0.7832 - val_accuracy: 0.8527 - lr: 0.0010
Epoch 3/25
7/7 [==============================] - ETA: 0s - loss: 0.1237 - accuracy: 0.9679
Epoch 3: val_loss did not improve from 0.78321


...

7/7 [==============================] - 6s 940ms/step - loss: 0.2593 - accuracy: 0.9128 - val_loss: 1.6305 - val_accuracy: 0.5670 - lr: 0.0010
Epoch 9/25
7/7 [==============================] - ETA: 0s - loss: 0.1857 - accuracy: 0.9404
Epoch 9: val_loss did not improve from 0.78321


7/7 [==============================] - 5s 694ms/step - loss: 0.1857 - accuracy: 0.9404 - val_loss: 2.3196 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 10/25
7/7 [==============================] - ETA: 0s - loss: 0.0967 - accuracy: 0.9777
Epoch 10: val_loss did not improve from 0.78321


7/7 [==============================] - 5s 705ms/step - loss: 0.0967 - accuracy: 0.9777 - val_loss: 5.0817 - val_accuracy: 0.4286 - lr: 0.0010
Epoch 11/25
7/7 [==============================] - ETA: 0s - loss: 0.1223 - accuracy: 0.9817
Epoch 11: val_loss did not improve from 0.78321


7/7 [==============================] - 7s 1s/step - loss: 0.1223 - accuracy: 0.9817 - val_loss: 3.6055 - val_accuracy: 0.5402 - lr: 0.0010
Epoch 12/25
7/7 [==============================] - ETA: 0s - loss: 0.1229 - accuracy: 0.9725Restoring model weights from the end of the best epoch: 2.

Epoch 12: val_loss did not improve from 0.78321


7/7 [==============================] - 8s 1s/step - loss: 0.1229 - accuracy: 0.9725 - val_loss: 3.7651 - val_accuracy: 0.5446 - lr: 0.0010
Epoch 12: early stopping


In [28]:
model.save('/content/gdrive/MyDrive/Autism_model_training/autistic_detection_by_emotion/test.h5')